In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
import json
import os
import numpy as np
import shutil # For sorting files

In [2]:
!kaggle --version


Kaggle API 1.7.4.5


In [3]:
os.makedirs("../data/user2", exist_ok=True)


In [5]:
# Download competition dataset into that folder (must palce kaggle.json file downlaoded from kaggle inside .kaggle folder)
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition -p ../data/user2


  0%|          | 0.00/814M [00:00<?, ?B/s]
  5%|▌         | 43.0M/814M [00:00<00:01, 442MB/s]
 11%|█         | 87.0M/814M [00:00<00:01, 447MB/s]
 16%|█▌        | 130M/814M [00:00<00:01, 437MB/s] 
 21%|██        | 172M/814M [00:00<00:01, 390MB/s]
 26%|██▌       | 210M/814M [00:00<00:01, 358MB/s]
 30%|███       | 245M/814M [00:00<00:01, 360MB/s]
 35%|███▍      | 281M/814M [00:00<00:01, 365MB/s]
 39%|███▉      | 317M/814M [00:00<00:01, 367MB/s]
 44%|████▎     | 355M/814M [00:00<00:01, 361MB/s]
 48%|████▊     | 394M/814M [00:01<00:01, 374MB/s]
 53%|█████▎    | 431M/814M [00:01<00:01, 377MB/s]
 58%|█████▊    | 471M/814M [00:01<00:00, 383MB/s]
 63%|██████▎   | 512M/814M [00:01<00:00, 386MB/s]
 68%|██████▊   | 553M/814M [00:01<00:00, 384MB/s]
 73%|███████▎  | 595M/814M [00:01<00:00, 390MB/s]
 78%|███████▊  | 634M/814M [00:01<00:00, 395MB/s]
 83%|████████▎ | 673M/814M [00:01<00:00, 397MB/s]
 88%|████████▊ | 715M/814M [00:01<00:00, 407MB/s]
 93%|█████████▎| 755M/814M [00:02<00:00, 400MB/s]
 98

In [6]:
import zipfile
from pathlib import Path

# unzip main file
main_zip = Path("../data/user2/dogs-vs-cats-redux-kernels-edition.zip")
with zipfile.ZipFile(main_zip, 'r') as z:
    z.extractall("../data/user2")

# unzip training set
train_zip = Path("../data/user2/train.zip")
with zipfile.ZipFile(train_zip, 'r') as z:
    z.extractall("../data/user2/train_images")


In [7]:

print("Sorting User 2's images...")

RAW_DATA_DIR = '../data/user2/train_images/train'
SORTED_DATA_DIR = '../data/user2/train_sorted'

# Create class folders if they don't exist
os.makedirs(os.path.join(SORTED_DATA_DIR, 'cats'), exist_ok=True)
os.makedirs(os.path.join(SORTED_DATA_DIR, 'dogs'), exist_ok=True)

# List files
all_files = os.listdir(RAW_DATA_DIR)
print(f"Found {len(all_files)} files in train_images")

for f in all_files:
    src = os.path.join(RAW_DATA_DIR, f)

    # Skip directories
    if os.path.isdir(src):
        continue

    fname = f.lower()  # make case-insensitive

    if fname.startswith('cat'):
        shutil.move(src, os.path.join(SORTED_DATA_DIR, 'cats', f))
    elif fname.startswith('dog'):
        shutil.move(src, os.path.join(SORTED_DATA_DIR, 'dogs', f))
    else:
        print("⚠️ Unknown file skipped:", f)

print("Image sorting complete.")

Sorting User 2's images...
Found 25000 files in train_images
Image sorting complete.


In [8]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
DATA_DIR_USER2 = SORTED_DATA_DIR # Use the new sorted directory

datagen_u2 = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator_u2 = datagen_u2.flow_from_directory(
    DATA_DIR_USER2,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', # Changed to binary
    subset='training'
)
validation_generator_u2 = datagen_u2.flow_from_directory(
    DATA_DIR_USER2,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', # Changed to binary
    subset='validation'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [10]:
# --- ML Code: Test Model V1 ---
print("Testing model_v1 on User 2's validation data...")
try:
    model_v1 = tf.keras.models.load_model('../models/model_v1.h5')
    print("Loaded model_v1.h5")

    results_v1_on_u2_data = model_v1.evaluate(validation_generator_u2)

    test_metrics_v1_u2 = {
        'test_loss': results_v1_on_u2_data[0],
        'test_accuracy': results_v1_on_u2_data[1]
    }
except Exception as e:
    print(f"Error testing model_v1: {e}")
    test_metrics_v1_u2 = {'test_loss': 'Error', 'test_accuracy': 'Error'}


Testing model_v1 on User 2's validation data...
Loaded model_v1.h5
157/157 ━━━━━━━━━━━━━━━━━━━━ 69s 432ms/step - accuracy: 0.5606 - loss: 0.6990


In [11]:

os.makedirs('../results', exist_ok=True)

# Save test results
with open('../results/test_v1_user2.json', 'w') as f:
    json.dump(test_metrics_v1_u2, f, indent=4)

print("Saved test_v1_user2.json to results/")

print("✅ Phase 2, Step 4: Testing of model_v1 complete.")


Saved test_v1_user2.json to results/
✅ Phase 2, Step 4: Testing of model_v1 complete.
